In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as transforms_v2  # CutMix를 위한 v2 transforms 추가
import sys
import os
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from tools.tool import AccuracyEarlyStopping, WarmUpLR, SAM  # 수정된 AccuracyEarlyStopping 클래스 임포트
from models.resnet import resnet18

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="resnet18_cfc,lr=0.01,factor=0.5,SAM_standard")  

# WandB 설정
config = {
    "model": "resnet18",
    "batch_size": 128,
    "num_epochs": 300,
    "learning_rate": 0.01,
    "optimizer": "Adam",
    "seed": 2025,
    "deterministic": False,
    "patience": 30,  # early stopping patience
    "max_epochs_wait": float('inf'),  # 최대 30 에폭까지만 기다림
    "cutmix_alpha": 1.0,  # CutMix 알파 파라미터 추가
    "cutmix_prob": 0.5,   # CutMix 적용 확률 추가
    "crop_padding": 4,    # RandomCrop 패딩 크기
    "crop_size": 32,      # RandomCrop 크기 (CIFAR-100 이미지 크기는 32x32)
    "warmup_epochs": 5,   # 웜업할 에폭 수 추가
}
wandb.config.update(config)

# CIFAR-100 데이터셋 로드 - 기본 train/test 분할 사용
transform_train = transforms.Compose([
    transforms.RandomCrop(config["crop_size"], padding=config["crop_padding"]),  # 패딩 후 랜덤 크롭
    transforms.RandomHorizontalFlip(),  # 수평 뒤집기
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

# DataLoader 생성
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=16)
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=16)

print(f"Train set size: {len(trainset)}")
print(f"Test set size: {len(testset)}")

# CutMix 변환 정의
cutmix = transforms_v2.CutMix(alpha=config["cutmix_alpha"], num_classes=100)  # CIFAR-100은 100개 클래스

def train(model, trainloader, criterion, optimizer, device, epoch, warmup_scheduler=None, warmup_epochs=5):
    """
    학습 함수 (CutMix 적용)
    """
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # CutMix 확률적 적용
        if random.random() < config["cutmix_prob"]:
            inputs, labels = cutmix(inputs, labels)
            # 이 경우 labels은 원-핫 인코딩 형태로 변환됨
            use_cutmix = True
        else:
            use_cutmix = False
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # CutMix 적용 여부에 따라 손실 함수 선택
        if use_cutmix:
            # CutMix가 적용된 경우 (원-핫 인코딩된 레이블)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            # 일반적인 경우 (정수 인덱스 레이블)
            loss = criterion(outputs, labels)
            
        loss.backward()
        optimizer.first_step(zero_grad=True)


        
        # 두 번째 forward-backward 패스
        if use_cutmix:
            # 새로운 forward 패스 필요
            outputs = model(inputs)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            # 새로운 forward 패스 필요
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.second_step(zero_grad=True)

        # 학습률 스케줄러 업데이트 - warmup 스케줄러만 여기서 업데이트
        if epoch < warmup_epochs and warmup_scheduler is not None:
            warmup_scheduler.step()
        
        running_loss += loss.item()

        # 정확도 계산 - CutMix 적용 여부에 따라 다르게 처리
        if use_cutmix:
            # 원-핫 인코딩된 레이블에서 argmax를 사용해 가장 큰 값의 인덱스 추출
            _, label_idx = labels.max(1)
        else:
            # 정수 인덱스 레이블 그대로 사용
            label_idx = labels
        
    
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += inputs.size(0)
        correct_top1 += predicted.eq(label_idx).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += sum([1 for i in range(len(label_idx)) if label_idx[i] in top5_idx[i]])
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, dataloader, criterion, device, epoch, phase="test"):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    eval_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            eval_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    eval_loss = eval_loss / len(dataloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'{phase.capitalize()} set: Epoch: {epoch+1}, Average loss:{eval_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return eval_loss, accuracy_top1, accuracy_top5

# 메인 학습 루프
def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs, patience, max_epochs_wait, warmup_scheduler=None, main_scheduler=None, warmup_epochs=5):
    """
    메인 학습 루프 (accuracy 기준 early stopping)
    """
    # 정확도 기반 얼리 스토핑 사용
    early_stopping = AccuracyEarlyStopping(patience=patience, verbose=True, path='checkpoint.pt', max_epochs=max_epochs_wait)
    
    best_test_acc_top1 = 0.0
    best_test_acc_top5 = 0.0
    
    # 테스트 정확도 기록을 위한 리스트
    test_acc_top1_history = []
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(
            model, 
            trainloader, 
            criterion, 
            optimizer, 
            device, 
            epoch, 
            warmup_scheduler, 
            warmup_epochs
        )
        
        # 테스트 데이터로 평가
        test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, epoch, phase="test")

        # 웜업 이후 ReduceLROnPlateau 스케줄러 업데이트 
        if epoch >= warmup_epochs and main_scheduler is not None:
            main_scheduler.step(test_acc_top1)  # 테스트 정확도에 따라 학습률 업데이트       
            
        # 테스트 정확도 기록
        test_acc_top1_history.append(test_acc_top1)
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "test_loss": test_loss,
            "test_accuracy_top1": test_acc_top1,
            "test_accuracy_top5": test_acc_top5
        })
            
        # 최고 정확도 모델 저장 (top-1 기준)
        if test_acc_top1 > best_test_acc_top1:
            best_test_acc_top1 = test_acc_top1
            best_test_acc_top5_at_best_top1 = test_acc_top5
            print(f'새로운 최고 top-1 정확도: {best_test_acc_top1:.2f}%, top-5 정확도: {best_test_acc_top5_at_best_top1:.2f}%')
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if test_acc_top5 > best_test_acc_top5:
            best_test_acc_top5 = test_acc_top5
            print(f'새로운 최고 top-5 정확도: {best_test_acc_top5:.2f}%')

        # Early stopping 체크 (test_acc_top1 기준)
        early_stopping(test_acc_top1, model, epoch)
        if early_stopping.early_stop:
            print(f"에폭 {epoch+1}에서 학습 조기 종료. 최고 성능 에폭: {early_stopping.best_epoch+1}")
            break
    
    # 훈련 완료 후 최고 모델 로드
    print("테스트 정확도 기준 최고 모델 로드 중...")
    model_path = f'best_model_{wandb.run.name}.pth'
    model.load_state_dict(torch.load(model_path))

    # 최종 테스트 세트 평가
    final_test_loss, final_test_acc_top1, final_test_acc_top5 = evaluate(model, testloader, criterion, device, num_epochs-1, phase="test")
    
    print(f'완료! 최고 테스트 top-1 정확도: {best_test_acc_top1:.2f}%, 최고 테스트 top-5 정확도: {best_test_acc_top5:.2f}%')
    print(f'최종 테스트 top-1 정확도: {final_test_acc_top1:.2f}%, 최종 테스트 top-5 정확도: {final_test_acc_top5:.2f}%')
    
    # WandB에 최종 결과 기록
    wandb.run.summary["best_test_accuracy_top1"] = best_test_acc_top1
    wandb.run.summary["best_test_accuracy_top5"] = best_test_acc_top5
    wandb.run.summary["final_test_accuracy_top1"] = final_test_acc_top1
    wandb.run.summary["final_test_accuracy_top5"] = final_test_acc_top5

    # Early stopping 정보 저장
    if early_stopping.early_stop:
        wandb.run.summary["early_stopped"] = True
        wandb.run.summary["early_stopped_epoch"] = epoch+1
        wandb.run.summary["best_epoch"] = early_stopping.best_epoch+1
    else:
        wandb.run.summary["early_stopped"] = False


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화
# 또는 매개변수 커스터마이징
model = resnet18().to(device)
criterion = nn.CrossEntropyLoss()  # 기본 CrossEntropyLoss 사용 (라벨 스무딩 없음)
base_optimizer = optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=config["learning_rate"])

# WarmUpLR 스케줄러 초기화
# 웜업할 총 iteration 수 계산 (웜업 에폭 × 배치 수)
warmup_steps = config["warmup_epochs"] * len(trainloader)
warmup_scheduler = WarmUpLR(optimizer, total_iters=warmup_steps)

# 웜업 이후 사용할 스케줄러 설정 
main_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',           # 정확도를 모니터링하므로 'max' 모드 사용
    factor=0.5,           # 학습률 감소 비율 (5배 감소)
    patience=5,           # 몇 에폭 동안 개선이 없을 때 감소시킬지
    verbose=True,         # 학습률 변경 시 출력
    threshold=0.01,        # 개선으로 간주할 최소 변화량
    min_lr=1e-6
)

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)

# 훈련 시작 시간 기록
start_time = time.time()

# 메인 학습 루프 호출
main_training_loop(
    model=model,
    trainloader=trainloader,
    testloader=testloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=config["num_epochs"],
    patience=config["patience"],
    max_epochs_wait=config["max_epochs_wait"],
    warmup_scheduler=warmup_scheduler,
    main_scheduler=main_scheduler,
    warmup_epochs=config["warmup_epochs"]
)

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"전체 학습 시간: {total_time:.2f} 초")

# WandB 실행 종료
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Files already downloaded and verified
Files already downloaded and verified
Train set size: 50000
Test set size: 10000
Using device: cuda
2개의 GPU를 사용합니다.

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  0%|                                                                                                       | 0/300 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch [1], Batch [50/391], Loss: 4.5961, LR: 0.000256
Epoch [1], Batch [100/391], Loss: 4.5056, LR: 0.000512
Epoch [1], Batch [150/391], Loss: 4.0830, LR: 0.000767
Epoch [1], Batch [200/391], Loss: 4.3724, LR: 0.001023
Epoch [1], Batch [250/391], Loss: 3.8148, LR: 0.001279
Epoch [1], Batch [300/391], Loss: 3.7220, LR: 0.001535
Epoch [1], Batch [350/391], Loss: 4.3313, LR: 0.001790
Train set: Epoch: 1, Average loss:4.2361, LR: 0.002000 Top-1 Accuracy: 6.3880%, Top-5 Accuracy: 22.4320%, Time consumed:91.87s
Test set: Epoch: 1, Average loss:3.7305, Top-1 Accuracy: 12.6700%, Top-5 Accuracy: 37.2200%, Time consumed:8.16s

새로운 최고 top-1 정확도: 12.67%, top-5 정확도: 37.22%
새로운 최고 top-5 정확도: 37.22%
Accuracy improved (-inf% --> 12.67%). Saving model ...


...

Epoch [25], Batch [50/391], Loss: 0.5745, LR: 0.010000
Epoch [25], Batch [100/391], Loss: 0.5118, LR: 0.010000
Epoch [25], Batch [150/391], Loss: 0.5782, LR: 0.010000
Epoch [25], Batch [200/391], Loss: 0.4586, LR: 0.010000
Epoch [25], Batch [250/391], Loss: 0.5844, LR: 0.010000
Epoch [25], Batch [300/391], Loss: 2.6163, LR: 0.010000
Epoch [25], Batch [350/391], Loss: 0.6092, LR: 0.010000
Train set: Epoch: 25, Average loss:1.5804, LR: 0.010000 Top-1 Accuracy: 67.3440%, Top-5 Accuracy: 88.2660%, Time consumed:94.32s
Test set: Epoch: 25, Average loss:1.1893, Top-1 Accuracy: 67.9000%, Top-5 Accuracy: 90.0100%, Time consumed:8.74s

새로운 최고 top-1 정확도: 67.90%, top-5 정확도: 90.01%
Accuracy improved (67.88% --> 67.90%). Saving model ...


  8%|███████▌                                                                                   | 25/300 [41:40<7:42:35, 100.93s/it]

Epoch [26], Batch [50/391], Loss: 0.4140, LR: 0.010000
Epoch [26], Batch [100/391], Loss: 0.5415, LR: 0.010000
Train set: Epoch: 26, Average loss:1.3923, LR: 0.010000 Top-1 Accuracy: 70.9580%, Top-5 Accuracy: 90.4360%, Time consumed:91.79s
Test set: Epoch: 26, Average loss:1.1603, Top-1 Accuracy: 68.3500%, Top-5 Accuracy: 90.7400%, Time consumed:8.17s

새로운 최고 top-1 정확도: 68.35%, top-5 정확도: 90.74%
새로운 최고 top-5 정확도: 90.74%
Accuracy improved (67.90% --> 68.35%). Saving model ...


  9%|███████▉                                                                                   | 26/300 [43:20<7:39:55, 100.71s/it]

Epoch [27], Batch [50/391], Loss: 0.9763, LR: 0.010000
Epoch [27], Batch [100/391], Loss: 0.3900, LR: 0.010000
